In [1]:
import psycopg2

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Database Connection

In [2]:
conn = psycopg2.connect(
    host='ruby.db.elephantsql.com',
    user ='fvhavaif',
    password='THCA_nW8eWwmkuQ4mkobpS0qvZNLEYzE',
    database='fvhavaif'
)

-------

# Trigger 1 

Cada vez que entre un cliente conocido, se debe agregar 1 punto a su estado de cuenta del
programa de fidelidad.

In [ ]:
def createTriggerAddPointWhenClientVisit():
    query = """
      CREATE OR REPLACE FUNCTION addPointsWhenClientVisit()
        RETURNS TRIGGER AS $$
          BEGIN
            IF NEW.idcliente IN (
                SELECT club_clientes.id_cliente
                FROM club_clientes
                WHERE  id_cliente=NEW.idcliente
            )
            THEN
                UPDATE club_clientes
                SET puntos = (puntos + 1)
                WHERE id_cliente = NEW.idcliente;
            END IF;
            RETURN NEW;
          END;

        $$ LANGUAGE plpgsql;
    ;"""

    cur = conn.cursor()
    cur.execute(query)
    cur.close()
    conn.commit()

    query = """
      CREATE TRIGGER addPointsWhenClientVisit
        AFTER INSERT ON visitas
        FOR EACH ROW
          EXECUTE PROCEDURE addPointsWhenClientVisit();  
    """
    
    cur = conn.cursor()
    cur.execute(query)
    cur.close()
    conn.commit()

createTriggerAddPointWhenClientVisit()




---

# Trigger 2

Cada vez que un cliente conocido realice una compra, se le agregaran los puntos en su programa de fidelidad


In [ ]:
def createTriggerAddPointsIfClientBuy():
    query = """
      CREATE OR REPLACE FUNCTION addPointsIfClientBuy()
        RETURNS TRIGGER AS $$
          DECLARE
            _date DATE := (
                SELECT MAX(fecha) FROM precio
                WHERE idproducto=NEW.id_prod
            )::date;

            precio NUMERIC := (
                SELECT precio.monto FROM precio
                WHERE precio.idproducto = NEW.id_prod
                AND fecha = _date
            );

            idClient NUMERIC := (
                SELECT idcliente FROM factura
                WHERE id=NEW.id_factura
            );

          BEGIN
            IF NEW.id_factura IN (
                SELECT factura.id FROM factura, club_clientes
                WHERE factura.idcliente = club_clientes.id_cliente
            ) THEN
                UPDATE club_clientes
                SET puntos = CAST((puntos + ((precio*NEW.cant) * 0.10)) as INT)
                WHERE id_cliente = idClient;
            END IF;
            RETURN NEW;
          END;

        $$ LANGUAGE plpgsql;
    """

    cur = conn.cursor()
    cur.execute(query)
    cur.close()
    conn.commit()

    query = """
      CREATE TRIGGER addPointsIfClientBuy
        AFTER INSERT ON ventas
        FOR EACH ROW
          EXECUTE PROCEDURE addPointsIfClientBuy();
    """
    
    cur = conn.cursor()
    cur.execute(query)
    cur.close()
    conn.commit()

createTriggerAddPointsIfClientBuy()



---
# Trigger 3 
Si un cliente desconocido con la misma cedula compra mas de 4 veces, automáticamente se agregará al programa de fidelidad.

In [ ]:
def addNewMember():
    query = """
      CREATE OR REPLACE FUNCTION addNewMember()
          RETURNS trigger AS $$
              DECLARE
                  purchases numeric := (SELECT COUNT(cliente.cedula) FROM cliente, factura
                  WHERE cliente.id = factura.idcliente);
              BEGIN
                  IF NEW.idcliente NOT IN (
                      SELECT id_cliente FROM club_clientes
                      WHERE id_cliente = NEW.idcliente
                  )
                  THEN
                      IF  (purchases > 4)
                      THEN
                          INSERT INTO club_clientes(id_cliente, puntos, visitas)
                          VALUES (NEW.idcliente, 0, purchases);
                      END IF;
                  END IF;
                  RETURN NEW;
              END;
      $$ LANGUAGE plpgsql
    ;"""

    cur = conn.cursor()
    cur.execute(query)
    cur.close()
    conn.commit()

    query = """
      CREATE TRIGGER addNewMember
        AFTER INSERT ON factura
        FOR EACH ROW
          EXECUTE PROCEDURE addNewMember()
    ;"""
    
    cur = conn.cursor()
    cur.execute(query)
    cur.close()
    conn.commit()

addNewMember()